In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add, Activation, Concatenate, Conv2D, Dropout 
from tensorflow.keras.layers import Flatten, Input, GlobalAveragePooling2D, MaxPooling2D
import tensorflow.keras.backend as K

import numpy as np

2024-04-30 10:47:29.571194: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 10:47:29.573697: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 10:47:29.602701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 10:47:30.115508: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:

def SqueezeNet(input_shape, nb_classes, use_bypass=False, dropout_rate=None, compression=1.0):
    
    input_img = Input(shape=input_shape)

    x = Conv2D(int(96*compression), (7,7), activation='relu', strides=(2,2), padding='same', name='conv1')(input_img)

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3', use_bypass=use_bypass)
    x = create_fire_module(x, int(32*compression), name='fire4')
    
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool4')(x)
    
    x = create_fire_module(x, int(32*compression), name='fire5', use_bypass=use_bypass)
    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7', use_bypass=use_bypass)
    x = create_fire_module(x, int(64*compression), name='fire8')
    
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool8')(x)
    
    x = create_fire_module(x, int(64*compression), name='fire9', use_bypass=use_bypass)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)
        
    x = output(x, nb_classes)

    return Model(inputs=input_img, outputs=x)


def SqueezeNet_11(input_shape, nb_classes, dropout_rate=None, compression=1.0):
        
    # input_img = Input(shape=input_shape)

    x = Conv2D(int(64*compression), (3,3), activation='relu', strides=(2,2), padding='same', name='conv1')(input_shape)

    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool1')(x)
    
    x = create_fire_module(x, int(16*compression), name='fire2')
    x = create_fire_module(x, int(16*compression), name='fire3')
    
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool3')(x)
    
    x = create_fire_module(x, int(32*compression), name='fire4')
    x = create_fire_module(x, int(32*compression), name='fire5')
    
    x = MaxPooling2D(pool_size=(3,3), strides=(2,2), name='maxpool5')(x)
    
    x = create_fire_module(x, int(48*compression), name='fire6')
    x = create_fire_module(x, int(48*compression), name='fire7')
    x = create_fire_module(x, int(64*compression), name='fire8')
    x = create_fire_module(x, int(64*compression), name='fire9')

    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    
    # Creating last conv10
    x = output(x, nb_classes)

    return Model(input_shape=(224,224,3), outputs=x)


def output(x, nb_classes):
    x = Conv2D(nb_classes, (1,1), strides=(1,1), padding='valid', name='conv10')(x)
    x = GlobalAveragePooling2D(name='avgpool10')(x)
    x = Activation("softmax", name='softmax')(x)
    return x


def create_fire_module(x, nb_squeeze_filter, name, use_bypass=False):
        
    nb_expand_filter = 4 * nb_squeeze_filter
    squeeze    = Conv2D(nb_squeeze_filter,(1,1), activation='relu', padding='same', name='%s_squeeze'%name)(x)
    expand_1x1 = Conv2D(nb_expand_filter, (1,1), activation='relu', padding='same', name='%s_expand_1x1'%name)(squeeze)
    expand_3x3 = Conv2D(nb_expand_filter, (3,3), activation='relu', padding='same', name='%s_expand_3x3'%name)(squeeze)
    
    axis = get_axis()
    x_ret = Concatenate(axis=axis, name='%s_concatenate'%name)([expand_1x1, expand_3x3])
    
    if use_bypass:
        x_ret = Add(name='%s_concatenate_bypass'%name)([x_ret, x])
        
    return x_ret


def get_axis():
    axis = -1 if K.image_data_format() == 'channels_last' else 1
    return axis

In [11]:
model=SqueezeNet(input_shape=(224,224,3),nb_classes=1000,dropout_rate=0.5,use_bypass=True)
tf.keras.utils.plot_model(model, to_file='test_squeeznet_skip_model.png', show_shapes=True)

model.summary()
# for lay in model1.layers:
#     print(lay.name)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 112, 112,  │     14,208 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool1            │ (None, 55, 55,    │          0 │ conv1[0][0]       │
│ (MaxPooling2D)      │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire2_squeeze       │ (None, 55, 55,    │      1,552 │ maxpool1[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire2_expand_1x1    │ (None, 55, 55,    │      1,088 │ fire2_squeeze[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire2_expand_3x3    │ (None, 55, 55,    │      9,280 │ fire2_squeeze[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire2_concatenate   │ (None, 55, 55,    │          0 │ fire2_expand_1x1… │
│ (Concatenate)       │ 128)              │            │ fire2_expand_3x3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire3_squeeze       │ (None, 55, 55,    │      2,064 │ fire2_concatenat… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire3_expand_1x1    │ (None, 55, 55,    │      1,088 │ fire3_squeeze[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire3_expand_3x3    │ (None, 55, 55,    │      9,280 │ fire3_squeeze[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire3_concatenate   │ (None, 55, 55,    │          0 │ fire3_expand_1x1… │
│ (Concatenate)       │ 128)              │            │ fire3_expand_3x3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire3_concatenate_… │ (None, 55, 55,    │          0 │ fire3_concatenat… │
│ (Add)               │ 128)              │            │ fire2_concatenat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire4_squeeze       │ (None, 55, 55,    │      4,128 │ fire3_concatenat… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire4_expand_1x1    │ (None, 55, 55,    │      4,224 │ fire4_squeeze[0]… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire4_expand_3x3    │ (None, 55, 55,    │     36,992 │ fire4_squeeze[0]… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fire4_concatenate   │ (None, 55, 55,    │          0 │ fire4_expand_1x1… │
│ (Concatenate)       │ 256)              │            │ fire4_expand_3x3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool4            │ (None, 27, 27,    │          0 │ fire4_concatenat

 Total params: 1,248,424 (4.76 MB)

 Trainable params: 1,248,424 (4.76 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
def generate_random_data(num_samples, input_shape):
    images = np.random.rand(num_samples, *input_shape)  # Random images
    labels = labels=np.random.dirichlet(np.ones(1000),size=num_samples)            # Random labels (steering angles)
    return images, labels

In [13]:
# Generate random training data
num_train_samples = 1000
input_shape = (224,224,3)
train_images, train_labels = generate_random_data(num_train_samples, input_shape)

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [15]:
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 9.9908e-07 - mae: 7.3574e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 9.9977e-07 - mae: 7.3622e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 1.0001e-06 - mae: 7.3616e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 1.0000e-06 - mae: 7.3610e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 9.9791e-07 - mae: 7.3616e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 9.9878e-07 - mae: 7.3565e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 1.0029e-06 - mae: 7.3689e-04 - val_loss: 1.0018e-06 - val_mae: 7.3671e-04
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 9.9921e-07 - mae: 

In [16]:
model.save_weights('./squeeznet.weights.h5')
model.save('./squeeznet_model.h5')
model.save('./squeeznet_model.keras')